<a href="https://colab.research.google.com/github/matiasdperez/Trabajo-Final-CV/blob/main/Trabajo_Final_CNN_Style_Transfer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Style Transfer

<img src="https://i0.wp.com/chelseatroy.com/wp-content/uploads/2018/12/neural_style_transfer.png?resize=768%2C311&ssl=1">

La idea de este trabajo final es reproducir el siguiente paper:

https://arxiv.org/pdf/1508.06576.pdf

El objetivo es transferir el estilo de una imagen dada a otra imagen distinta. 

Como hemos visto en clase, las primeras capas de una red convolucional se activan ante la presencia de ciertos patrones vinculados a detalles muy pequeños.

A medida que avanzamos en las distintas capas de una red neuronal convolucional, los filtros se van activando a medida que detectan patrones de formas cada vez mas complejos.

Lo que propone este paper es asignarle a la activación de las primeras capas de una red neuronal convolucional (por ejemplo VGG19) la definición del estilo y a la activación de las últimas capas de la red neuronal convolucional, la definición del contenido.

La idea de este paper es, a partir de dos imágenes (una que aporte el estilo y otra que aporte el contenido) analizar cómo es la activación de las primeras capas para la imagen que aporta el estilo y cómo es la activación de las últimas capas de la red convolucional para la imagen que aporta el contenido. A partir de esto se intentará sintetizar una imagen que active los filtros de las primeras capas que se activaron con la imagen que aporta el estilo y los filtros de las últimas capas que se activaron con la imagen que aporta el contenido.

A este procedimiento se lo denomina neural style transfer.

# En este trabajo se deberá leer el paper mencionado y en base a ello, entender la implementación que se muestra a continuación y contestar preguntas sobre la misma.

# Una metodología posible es hacer una lectura rápida del paper (aunque esto signifique no entender algunos detalles del mismo) y luego ir analizando el código y respondiendo las preguntas. A medida que se planteen las preguntas, volviendo a leer secciones específicas del paper terminará de entender los detalles que pudieran haber quedado pendientes.

Lo primero que haremos es cargar dos imágenes, una que aporte el estilo y otra que aporte el contenido. A tal fin utilizaremos imágenes disponibles en la web.

In [ ]:
! pip install tensorflow==1.14

In [1]:
# Imagen para estilo
!wget https://upload.wikimedia.org/wikipedia/commons/5/52/La_noche_estrellada1.jpg

# Imagen para contenido
!wget https://upload.wikimedia.org/wikipedia/commons/thumb/f/f4/Neckarfront_T%C3%BCbingen_Mai_2017.jpg/775px-Neckarfront_T%C3%BCbingen_Mai_2017.jpg

# Creamos el directorio para los archivos de salida
!mkdir /content/output

--2022-07-27 21:13:38--  https://upload.wikimedia.org/wikipedia/commons/5/52/La_noche_estrellada1.jpg
Resolving upload.wikimedia.org (upload.wikimedia.org)... 208.80.154.240, 2620:0:861:ed1a::2:b
Connecting to upload.wikimedia.org (upload.wikimedia.org)|208.80.154.240|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 223725 (218K) [image/jpeg]
Saving to: ‘La_noche_estrellada1.jpg’

La_noche_estrellada 100%[===================>] 218.48K  --.-KB/s    in 0.007s  

2022-07-27 21:13:38 (29.3 MB/s) - ‘La_noche_estrellada1.jpg’ saved [223725/223725]

--2022-07-27 21:13:38--  https://upload.wikimedia.org/wikipedia/commons/thumb/f/f4/Neckarfront_T%C3%BCbingen_Mai_2017.jpg/775px-Neckarfront_T%C3%BCbingen_Mai_2017.jpg
Resolving upload.wikimedia.org (upload.wikimedia.org)... 208.80.154.240, 2620:0:861:ed1a::2:b
Connecting to upload.wikimedia.org (upload.wikimedia.org)|208.80.154.240|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 153015 (149K) [ima

In [14]:
import tensorflow as tf
from tensorflow.keras import preprocessing, applications, backend
#from keras.preprocessing.image import load_img, save_img, img_to_array
import numpy as np
from scipy.optimize import fmin_l_bfgs_b
import time
import argparse
import tensorflow as tf

#from keras.applications import vgg19
#from keras import backend as K
from pathlib import Path
from PIL import Image

In [3]:
# Definimos las imagenes que vamos a utilizar, y el directorio de salida

base_image_path = Path("/content/775px-Neckarfront_Tübingen_Mai_2017.jpg")
style_reference_image_path = Path("/content/La_noche_estrellada1.jpg")
result_prefix = Path("/content/output")
iterations = 100

# 1) En base a lo visto en el paper ¿Qué significan los parámetros definidos en la siguiente celda?

Respuesta: los parámetros mostrados a continuación se corresponden con las contribuciones (*weights*) de cada una de las losses (*content loss*, *style loss* y *total variation loss*) en la loss final a optimizar. En otras palabras, la loss es una suma ponderada de términos:


*   *Content loss*: mide la distancia L2 entre los features maps de la imagen que aporta el contenido y la imagen combinada, generados por alguna de las últimas capas de la convnet. Se utilizan las activaciones de una capa profunda en la red ya que estas capturan aspectos generales y/o abstractos de la imagen.
*  *Style loss*: mide cuan similares son en estilo la imagen generada y la imagen que aporta el estilo, para un conjunto de capas de activación a distintos niveles de profundidad de la convnet. El "estilo" es capturado a partir de la matriz de correlaciones (o matriz de Gram) de los features maps.
*   *Totsl Variation loss*: funciona como regularización del modelo para evitar generar una imagen muy pixelada.






In [6]:
total_variation_weight = 0.1
style_weight = 10
content_weight = 1

In [7]:
# Definimos el tamaño de las imágenes a utilizar
width, height = preprocessing.image.load_img(base_image_path).size
img_nrows = 400
img_ncols = int(width * img_nrows / height)

In [31]:
img = preprocessing.image.load_img(base_image_path, target_size=(img_nrows, img_ncols))

In [33]:
img = applications.vgg19.preprocess_input(np.expand_dims(img, axis=0))

In [34]:
img = img.reshape((img_nrows, img_ncols, 3))

# 2) Explicar qué hace la siguiente celda. En especial las últimas dos líneas de la función antes del return. ¿Por qué?

Ayuda: https://keras.io/applications/

Respuesta: la función tiene por parámetro al path de una imagen y preprocesará la misma de manera secuncial de la siguiente manera:


1.   Carga la imagen en memoria a partir del path dado y ajusta su tamaño de acuerdo al tamaño definido en target_size.
2.   Transforma la imagen de formato PIL a un array de tamaño 400 x 517 x 3 canales
3.  Agrega una dimensión adicional al array la cual refiere al tamaño del batch: 1 batch_size x 400 x 517 x 3 canales. Este cambio de dimensiones busca adecuar el formato de la imagen al input esperado por la red de tipo VGG19.
4.   Preprocesa/normaliza la imagen: primero convierte el formato de RGB a BGR y luego centra los valores de cada canal en 0 restándoles las medias de cada canal en base al dataset ImageNet, sin escalarlo.







In [12]:
def preprocess_image(image_path):
    img = preprocessing.image.load_img(image_path, target_size=(img_nrows, img_ncols))
    img = preprocessing.image.img_to_array(img)
    img = np.expand_dims(img, axis=0)
    img = applications.vgg19.preprocess_input(img)
    return img

# 3) Habiendo comprendido lo que hace la celda anterior, explique de manera muy concisa qué hace la siguiente celda. ¿Qué relación tiene con la celda anterior?

Respuesta: esta función revierte el preprocesamiento hecho con la función anterior:

1.   Transforma al tensor 4D en 3D, eliminando la dimension del batch_size
2.   Revierte el centrado en 0 sumandole la media de cada canal de acuerdo al dataset ImageNet
3.   Cambia el orden de los canales: en lugar de blue-green-red se pasa a red-green-blue
4.   Garantiza que los valores se encuentre entre 0 y 255



In [13]:
def deprocess_image(x):
    x = x.reshape((img_nrows, img_ncols, 3))
    # Remove zero-center by mean pixel
    x[:, :, 0] += 103.939
    x[:, :, 1] += 116.779
    x[:, :, 2] += 123.68
    # 'BGR'->'RGB'
    x = x[:, :, ::-1]
    x = np.clip(x, 0, 255).astype('uint8')
    return x

In [15]:
# get tensor representations of our images
# K.variable convierte un numpy array en un tensor, para 
base_image = backend.variable(preprocess_image(base_image_path))
style_reference_image = backend.variable(preprocess_image(style_reference_image_path))

In [16]:
combination_image = backend.placeholder((1, img_nrows, img_ncols, 3))

Aclaración:

La siguiente celda sirve para procesar las tres imagenes (contenido, estilo y salida) en un solo batch.

In [17]:
# combine the 3 images into a single Keras tensor
input_tensor = backend.concatenate([base_image,
                              style_reference_image,
                              combination_image], axis=0)

In [18]:
# build the VGG19 network with our 3 images as input
# the model will be loaded with pre-trained ImageNet weights
model = applications.vgg19.VGG19(input_tensor=input_tensor,
                    weights='imagenet', include_top=False)
print('Model loaded.')

# get the symbolic outputs of each "key" layer (we gave them unique names).
outputs_dict = dict([(layer.name, layer.output) for layer in model.layers]) # Contiene los campos de activación para cada capa y para cada imagen

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


80142336/80134624 [==============================] - 1s 0us/step
Model loaded.


# 4) En la siguientes celdas:

- ¿Qué es la matriz de Gram?¿Para qué se usa?

La matriz de Gram es una matriz simétrica de dimensión N, donde N es el número de feature maps o filtros para una capa específica de una convnet. Los elementos que la conforman son los productos punto de los feature_maps vectorizados (es decir, aplicando un flatten a cada feature map con dimensiones width x height x channels) y se asemejan a una correlación entre cada feature_map (sin centran ni promediar). Esta matriz captura el componente estilístico de la imagen: dado que cada feature_map se enfoca en una determinada característica de la imagen, un valor elevado del producto punto entre estos dos feature_maps indica que esas características suelen presentarse en conjunto en la imagen, frente a valores bajos que muestra que cierta combinación de patrones no esta presente en la imagen. Esta matriz se calcula para distitas layers, buscando capturar distintas texturas a diferentes escalas en lugar de la simple presencia o no de un objeto.

- ¿Por qué se permutan las dimensiones de x?

Las dimensiones del tensor se permutan antes de vectorizarlo para garantizar la conformidad de la matrices, esto es, para que puedan multiplicarse entre si. Esto permite que el producto punto entre filas y columnas se asimile a la correlación entre cada par de feature_maps.

In [24]:
def gram_matrix(x):
    features = backend.batch_flatten(backend.permute_dimensions(x, (2, 0, 1)))
    gram = backend.dot(features, backend.transpose(features))
    return gram

# 5) Losses:

Explicar qué mide cada una de las losses en las siguientes tres celdas.

Rta:

* Content loss: mide la distancia L2 entre los features maps de la imagen que 
aporta el contenido y la imagen combinada, generados por alguna de las últimas capas de la convnet. Se utilizan las activaciones de una capa profunda en la red ya que estas capturan la presencia de aspectos más generales de la imagen.

* Style loss: mide cuan similares son en estilo la imagen generada y la imagen que aporta el estilo, para un conjunto de capas de activación a distintos niveles de profundidad de la convnet. El "estilo" es capturado a partir de la matriz de correlaciones (o matriz de Gram) de los features maps.

* Total Variation loss: funciona como regularización del modelo para evitar generar una imagen muy pixelada. La misma se basa en la distancia euclidea pixel a pixel entre dos "recortes" de la imagen con un pixel de diferencia.

In [25]:
def style_loss(style, combination):
    assert backend.ndim(style) == 3
    assert backend.ndim(combination) == 3
    S = gram_matrix(style)
    C = gram_matrix(combination)
    channels = 3
    size = img_nrows * img_ncols
    return backend.sum(backend.square(S - C)) / (4.0 * (channels ** 2) * (size ** 2))

In [26]:
def content_loss(base, combination):
    return backend.sum(backend.square(combination - base))


In [27]:
def total_variation_loss(x):
    assert backend.ndim(x) == 4
    a = backend.square(
        x[:, :img_nrows - 1, :img_ncols - 1, :] - x[:, 1:, :img_ncols - 1, :])
    b = backend.square(
        x[:, :img_nrows - 1, :img_ncols - 1, :] - x[:, :img_nrows - 1, 1:, :])
    return backend.sum(backend.pow(a + b, 1.25))


In [28]:
# Armamos la loss total
loss = backend.variable(0.0)
layer_features = outputs_dict['block5_conv2']
base_image_features = layer_features[0, :, :, :]
combination_features = layer_features[2, :, :, :]
loss = loss + content_weight * content_loss(base_image_features,
                                            combination_features)

feature_layers = ['block1_conv1', 'block2_conv1',
                  'block3_conv1', 'block4_conv1',
                  'block5_conv1']
for layer_name in feature_layers:
    layer_features = outputs_dict[layer_name]
    style_reference_features = layer_features[1, :, :, :] 
    combination_features = layer_features[2, :, :, :]
    sl = style_loss(style_reference_features, combination_features)
    loss = loss + (style_weight / len(feature_layers)) * sl
loss = loss + total_variation_weight * total_variation_loss(combination_image) #La función de pérdida está parametrizada para los campos de activación de las imagenes de contenido y estilo

In [29]:
grads = backend.gradients(loss, combination_image) #El gradiente de la función de pérdida es computado respecto a la variable 'combination_image'

outputs = [loss]
if isinstance(grads, (list, tuple)):
    outputs += grads
else:
    outputs.append(grads)

f_outputs = backend.function([combination_image], outputs)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


# 6) Explique el propósito de las siguientes tres celdas. ¿Qué hace la función fmin_l_bfgs_b? ¿En qué se diferencia con la implementación del paper? ¿Se puede utilizar alguna alternativa?

Respuesta:

In [24]:
def eval_loss_and_grads(x):
    x = x.reshape((1, img_nrows, img_ncols, 3))
    outs = f_outputs([x])
    loss_value = outs[0]
    if len(outs[1:]) == 1:
        grad_values = outs[1].flatten().astype('float64')
    else:
        grad_values = np.array(outs[1:]).flatten().astype('float64')
    return loss_value, grad_values

# this Evaluator class makes it possible
# to compute loss and gradients in one pass
# while retrieving them via two separate functions,
# "loss" and "grads". This is done because scipy.optimize
# requires separate functions for loss and gradients,
# but computing them separately would be inefficient.

In [25]:
class Evaluator(object):

    def __init__(self):
        self.loss_value = None
        self.grads_values = None

    def loss(self, x):
        assert self.loss_value is None
        loss_value, grad_values = eval_loss_and_grads(x)
        self.loss_value = loss_value
        self.grad_values = grad_values
        return self.loss_value

    def grads(self, x):
        assert self.loss_value is not None
        grad_values = np.copy(self.grad_values)
        self.loss_value = None
        self.grad_values = None
        return grad_values

# 7) Ejecute la siguiente celda y observe las imágenes de salida en cada iteración.

In [ ]:
evaluator = Evaluator()

# run scipy-based optimization (L-BFGS) over the pixels of the generated image
# so as to minimize the neural style loss
x = preprocess_image(base_image_path)

for i in range(iterations):
    print('Start of iteration', i)
    start_time = time.time()
    x, min_val, info = fmin_l_bfgs_b(evaluator.loss, x.flatten(),
                                     fprime=evaluator.grads, maxfun=20)
    print('Current loss value:', min_val)
    # save current generated image
    img = deprocess_image(x.copy())
    fname = result_prefix / ('output_at_iteration_%d.png' % i)
    tf.keras.preprocessing.image.save_img(fname, img)
    end_time = time.time()
    print('Image saved as', fname)
    print('Iteration %d completed in %ds' % (i, end_time - start_time))

Start of iteration 0
Current loss value: 13284553000.0
Image saved as /content/output/output_at_iteration_0.png
Iteration 0 completed in 345s
Start of iteration 1
Current loss value: 6357179000.0
Image saved as /content/output/output_at_iteration_1.png
Iteration 1 completed in 357s
Start of iteration 2
Current loss value: 4114816800.0
Image saved as /content/output/output_at_iteration_2.png
Iteration 2 completed in 340s
Start of iteration 3
Current loss value: 3166202400.0
Image saved as /content/output/output_at_iteration_3.png
Iteration 3 completed in 337s
Start of iteration 4
Current loss value: 2651358700.0
Image saved as /content/output/output_at_iteration_4.png
Iteration 4 completed in 328s
Start of iteration 5
Current loss value: 2326377700.0
Image saved as /content/output/output_at_iteration_5.png
Iteration 5 completed in 329s
Start of iteration 6
Current loss value: 2023148800.0
Image saved as /content/output/output_at_iteration_6.png
Iteration 6 completed in 331s
Start of ite

# 8) Generar imágenes para distintas combinaciones de pesos de las losses. Explicar las diferencias. (Adjuntar las imágenes generadas como archivos separados.)

Respuesta:

# 9) Cambiar las imágenes de contenido y estilo por unas elegidas por usted. Adjuntar el resultado.

Respuesta: